# Trying to ingest live data from ccxt 

Getting lost in bcolz process, so...

Settling for: https://github.com/enigmampc/catalyst/issues/65

Pulled __main__.py out of project to remove exchange limitations

Pulled exchange_bundle.py out of project to download data from ccxt api and create csv file before ingesting.

Using a command like: kryptobot ingest-exchange -x binance -f minute -i ltc_btc --csv none

Hope to refactor and remove the --csv parameter so it goes through the normal process, but creates csv files and switches to ingest those files.

In [ ]:
from kryptobot.catalyst_extensions.exchange.exchange_bundle import ExchangeBundle
from catalyst.exchange.bundle_utils import range_in_bundle

exchange_name = 'binance'
data_frequency = 'daily'
include_symbols = 'ltc_btc'

exchange_bundle = ExchangeBundle(exchange_name)
ingest = exchange_bundle.ingest(
    data_frequency=data_frequency,
    include_symbols=include_symbols,
#     exclude_symbols=params['exclude_symbols'],
#     start=start,
#     end=end,
    show_progress=True,
#     show_breakdown=params['show_breakdown'],
#     show_report=params['show_report'],
#     csv=params['csv']
)
print(ingest)

# catalyst ingest -b binance -c

In [17]:
import ccxt
import json
import pandas as pd

symbol = 'ltc_btc'
ccxt_symbol = symbol.replace('_', '/').upper()

exchange = ccxt.binance()
ohlcv = json.dumps(exchange.fetch_ohlcv(ccxt_symbol, '1d'))
# print(ohlcv)
raw = pd.read_json(ohlcv)
raw.columns = ['date','open','high','low','close','volume']
raw['date'] = pd.to_datetime(raw['date'], unit='ms')
raw.set_index('date', inplace=True)
scale = 1
raw.loc[:, 'open'] /= scale
raw.loc[:, 'high'] /= scale
raw.loc[:, 'low'] /= scale
raw.loc[:, 'close'] /= scale
raw.loc[:, 'volume'] *= scale
raw.reset_index(level=0, inplace=True)
raw.insert(0, 'symbol', symbol)
raw.tail()

,symbol,date,open,high,low,close,volume
339,ltc_btc,2018-06-18,0.014721,0.014855,0.014489,0.014722,141781.73
340,ltc_btc,2018-06-19,0.014720,0.015040,0.014516,0.014631,126201.45
341,ltc_btc,2018-06-20,0.014636,0.014750,0.014427,0.014460,123539.56
342,ltc_btc,2018-06-21,0.014465,0.014607,0.014300,0.014403,137659.67
343,ltc_btc,2018-06-22,0.014396,0.014427,0.013656,0.013976,128181.39


In [8]:
import bcolz
from catalyst.data.us_equity_pricing import BcolzDailyBarReader

# pricing_path = '/root/.catalyst/data/exchanges/poloniex/minute_bundle/00/33/003314.bcolz/'
# pricing_path = 'root/.catalyst/data/exchanges/bitfinex/temp_bundles/bitfinex-daily-avt_btc-2017/89/18/891835.bcolz/'
pricing_path = '/root/.catalyst/data/exchanges/bitfinex/daily_bundle/00/89/008966.bcolz/'
ct = bcolz.ctable(rootdir=pricing_path)
df = ct[["open", "high", "low", "close", "volume"]].todataframe()
df.tail()

# cls(BcolzDailyBarReader(pricing_path))

,open,high,low,close,volume
1200,148760,148760,148760,148760,2006590054
1201,146000,146000,133900,133900,17186497585
1202,133900,133900,108610,108610,4800000000
1203,108610,108610,108610,108610,0
1204,108610,108610,108610,108610,0
